In [13]:
import cv2
import os
import numpy as np
from skimage import io, color, filters, morphology

In [14]:
def zhang_suen_thinning(image):
    # Implementasi algoritma Zhang-Suen Thinning
    skeleton = np.zeros(image.shape, dtype=np.uint8)
    prev = np.zeros(image.shape, dtype=np.uint8)
    diff = None

    while True:
        prev[:] = skeleton
        for step in range(1, 3):
            for i in range(1, image.shape[0] - 1):
                for j in range(1, image.shape[1] - 1):
                    P2 = image[i-1, j]
                    P3 = image[i-1, j+1]
                    P4 = image[i, j+1]
                    P5 = image[i+1, j+1]
                    P6 = image[i+1, j]
                    P7 = image[i+1, j-1]
                    P8 = image[i, j-1]
                    P9 = image[i-1, j-1]
                    A = (P2 == 0 and P3 == 1) + (P3 == 0 and P4 == 1) + \
                        (P4 == 0 and P5 == 1) + (P5 == 0 and P6 == 1) + \
                        (P6 == 0 and P7 == 1) + (P7 == 0 and P8 == 1) + \
                        (P8 == 0 and P9 == 1) + (P9 == 0 and P2 == 1)
                    B = P2 + P3 + P4 + P5 + P6 + P7 + P8 + P9
                    m1 = step == 1 and (P2 * P4 * P6 == 0) and (P4 * P6 * P8 == 0)
                    m2 = step == 2 and (P2 * P4 * P8 == 0) and (P2 * P6 * P8 == 0)
                    if A == 1 and 2 <= B <= 6 and m1 and m2:
                        skeleton[i, j] = 0
                    else:
                        skeleton[i, j] = image[i, j]
        diff = np.sum(np.abs(skeleton - prev))
        if diff == 0:
            break
    return skeleton

In [15]:
# Direktori dataset asli
dataset_asli = "Dataset1/SUV"

# Direktori untuk menyimpan dataset hasil deteksi tepi
dataset_thinning = "dataset_thinning"

# Membuat direktori untuk menyimpan dataset hasil deteksi tepi jika belum ada
if not os.path.exists(dataset_thinning):
    os.makedirs(dataset_thinning)

In [16]:
# Process images
for label in os.listdir(dataset_asli):
    label_dir = os.path.join(dataset_asli)  # Construct correct path

    # Check if label directory exists before processing
    if os.path.isdir(label_dir):
        for filename in os.listdir(label_dir):
            image_path = os.path.join(label_dir, filename)
            image = cv2.imread(image_path)

            # Handle potential file reading errors
            if image is None:
                print(f"Error: Could not read image {image_path}")
                continue  # Skip to the next image

            # Convert to grayscale
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, binary = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY_INV)

            # Deteksi tepi menggunakan Canny
            edges = cv2.Canny(binary, 100, 200)

            # Thinning menggunakan implementasi Zhang-Suen
            thinned = zhang_suen_thinning(edges // 255) * 255

            # Save thinned image
            cv2.imwrite(os.path.join(dataset_thinning, filename), thinned)

KeyboardInterrupt: 